In [14]:
import requests
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup

In [6]:
data  = []
for i in tqdm(range(1,51)):
    link = 'https://books.toscrape.com/catalogue/page-' + str(i) + '.html'

    res = requests.get(link)
    soup = BeautifulSoup(res.text , 'html.parser')

    for sp in soup.find_all('li', class_ = 'col-xs-6 col-sm-4 col-md-3 col-lg-3'):

        # print("---->",sp)
        img_link    = 'https://books.toscrape.com/' + sp.find('img').get('src')[3:]
        book_link   = 'https://books.toscrape.com/catalogue/' + sp.find_all('a')[-1].get('href')
        title       =   sp.find_all('a')[-1].get('title')
        rating      =   sp.find('p').get('class')[-1]
        price       =   sp.find('p' , class_ = 'price_color').text[2:]
        stock       =   sp.find('p', class_ = 'instock availability').text.strip()

        data.append([title, rating, price, stock, book_link, img_link])

        df = pd.DataFrame(data, columns = ['title', 'rating', 'price', 'stock', 'book_link', 'img_link'])

        # df.head()

        df.to_csv('books.csv', index = False)

100%|██████████| 50/50 [01:02<00:00,  1.26s/it]


In [11]:
df = pd.read_csv('books.csv')

for link in tqdm(df['book_link']):
    
    res = requests.get(link)
    soup = BeautifulSoup(res.text, 'html.parser')
    typ = soup.find('ul', class_ = 'breadcrumb').find_all('a')[2].text
    temp = soup.find('table', class_ = 'table table-striped').find_all('td')

    upc      = temp[0].text
    price_x  = temp[2].text[2:]
    price_i  = temp[3].text[2:]
    tax      = temp[4].text[2:]
    qn       = temp[5].text
    reviews  = temp[6].text

    data.append([typ, price_x, price_i, tax, qn, upc, reviews])
    # print(soup)


df = pd.DataFrame(data, columns = ['category','price_e_tax', 'price_i_tax', 'tax', 'quantity', 'upc', 'reviews'])

df.head()

df.to_csv('data.csv', index = False)


  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 1000/1000 [17:06<00:00,  1.03s/it]


In [16]:
df_books = pd.read_csv('books.csv')
df_data = pd.read_csv('data.csv')

df = pd.DataFrame()

df['title']         = df_books['title']
df['upc']           = df_data['upc']
df['category']      = df_data['category']
df['price_e_tax']   = df_data['price_e_tax']
df['price_i_tax']   = df_data['price_i_tax']
df['tax']           = df_data['tax']
df['rating']        = df_books['rating']
df['reviews']       = df_data['reviews']
df['quantity']      = df_data['quantity']
df['stock']         = df_books['stock']

df['book_link']     = df_books['book_link']
df['img_link']      = df_books['img_link']

df.to_csv('final.csv',index = False)